<a href="https://colab.research.google.com/github/Henil21/Natural_Language_processing/blob/main/Hindi_sum_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
!pip install transformers datasets evaluate rouge_score

In [ ]:
!pip install -U accelerate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from datasets import load_dataset

billsum= load_dataset("Someman/hindi-summarization")
billsum

DatasetDict({
    train: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 55226
    })
    test: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 13807
    })
    valid: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 17265
    })
})

In [27]:
billsum_t=billsum['train'][:20000]

In [28]:
billsum_tst=billsum['test'][:5000]

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [30]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [38]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

AttributeError: ignored

In [32]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [34]:
import evaluate

rouge = evaluate.load("rouge")


In [35]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [36]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
tokenized_billsum["train"][:1000]

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="hindi_suml",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.216800,0.204213,0.009000,0.002300,0.009000,0.008900,19.000000


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,0.216800,0.204213,0.009000,0.002300,0.009000,0.008900,19.000000
2,0.210300,0.203622,0.017700,0.004200,0.017800,0.017800,19.000000
3,0.208200,0.200437,0.017800,0.004000,0.017800,0.017800,19.000000
